In [8]:
import numpy as np
import torch
import time
from torchvision.models import *
import pandas as pd
import os

In [9]:
# make models from str
model_name = "resnet18"
# make results
os.makedirs("results", exist_ok=True)

In [3]:
def computeTime(model, input_size=[1, 3, 224, 224], device='cuda', FP16=False):
    inputs = torch.randn(input_size)
    if device == 'cuda':
        model = model.cuda()
        inputs = inputs.cuda()
    if FP16:
        model = model.half()
        inputs = inputs.half()

    model.eval()

    i = 0
    time_spent = []
    while i < 200:
        start_time = time.time()
        with torch.no_grad():
            _ = model(inputs)

        if device == 'cuda':
            torch.cuda.synchronize() # wait for cuda to finish (cuda is asynchronous!)
        if i != 0:
            time_spent.append(time.time() - start_time)
        i += 1
    print('Avg execution time (ms): {:.3f}'.format(np.mean(time_spent)))
    return np.mean(time_spent)

In [4]:
modellist = ["resnet18", "resnet34", "resnet50", "resnet101", "resnet152", \
 "resnext50_32x4d", "resnext101_32x8d", "mnasnet1_0", "squeezenet1_0", "densenet121", "densenet169", "inception_v3"]

# resnet is enought for now
modellist = ["resnet18", "resnet34", "resnet50"]

In [5]:
batchlist = [1, 4, 8, 16, 32]
imsize = [128, 256, 512, 1024]

for i, model_name in enumerate(modellist):

    runtimes = []
    
    # define model
    print("model: {}".format(model_name))
    mdl = globals()[model_name]
    model = mdl()
    
    for batch in batchlist:        
        runtimes.append(computeTime(model, input_size=[batch, 3, 256, 256], device="cuda", FP16=False)/batch)

    if i == 0:
        dfbatch = pd.DataFrame({model_name: runtimes},
                         index = batchlist)
    else:
        dfbatch[model_name] = runtimes
        
    runtimes = []
    for isize in imsize:
        runtimes.append(computeTime(model, input_size=[1, 3, isize, isize], device="cuda", FP16=False))

    if i == 0:
        dfimsize = pd.DataFrame({model_name: runtimes},
                         index = imsize)
    else:
        dfimsize[model_name] = runtimes

model: resnet18
Avg execution time (ms): 0.009
Avg execution time (ms): 0.019
Avg execution time (ms): 0.047
Avg execution time (ms): 0.040
Avg execution time (ms): 0.091
Avg execution time (ms): 0.008
Avg execution time (ms): 0.008
Avg execution time (ms): 0.020
Avg execution time (ms): 0.072
model: resnet34
Avg execution time (ms): 0.015
Avg execution time (ms): 0.037
Avg execution time (ms): 0.079
Avg execution time (ms): 0.066
Avg execution time (ms): 0.146
Avg execution time (ms): 0.014
Avg execution time (ms): 0.015
Avg execution time (ms): 0.038
Avg execution time (ms): 0.135
model: resnet50
Avg execution time (ms): 0.018
Avg execution time (ms): 0.045
Avg execution time (ms): 0.093
Avg execution time (ms): 0.137
Avg execution time (ms): 0.279
Avg execution time (ms): 0.014
Avg execution time (ms): 0.018
Avg execution time (ms): 0.048
Avg execution time (ms): 0.179


In [13]:
dfbatch.to_csv("results/batch.csv")
dfimsize.to_csv("results/imsize.csv")
dfbatch

,resnet18,resnet34,resnet50
1,0.008542,0.014813,0.017847
4,0.004840,0.009199,0.011361
8,0.005878,0.009894,0.011598
16,0.002495,0.004145,0.008592
32,0.002836,0.004564,0.008728


In [14]:
dfimsize

,resnet18,resnet34,resnet50
128,0.007869,0.014414,0.014149
256,0.007717,0.014684,0.017656
512,0.020139,0.037699,0.047882
1024,0.071687,0.134825,0.179115


In [15]:
import 

ModuleNotFoundError: No module named 'apex'